In [1]:
import sys
sys.path.append('..')

In [23]:
import numpy as np
import torch
import torch.nn as nn
import collections
import torchvision
import torchvision.transforms as transforms

from PIL import Image
from pathlib import Path
from catalyst.data.augmentor import Augmentor
from catalyst.utils.factory import UtilsFactory
from catalyst.models.segmentation import UNet

In [3]:
np.random.seed(1488)
n_images = 500
bs = 4
n_workers = 4

In [4]:
images = [np.random.uniform(high=256, size=(512, 512)).astype(np.uint8)
          for x in range(n_images)]
masks = [(np.random.uniform(high=1, size=(512, 512)) > 0.5).astype(float)
         for x in range(n_images)]
data = list(zip(images, masks))

train_data = data[:n_images // 2]
valid_data = data[n_images // 2:]

In [5]:
# # Saving mock data
# data_dir = Path('..') / 'data'

# np.save(data_dir / 'mock_data_images', np.stack(images))
# np.save(data_dir / 'mock_data_masks', np.stack(masks))

# (data_dir / 'mock_data_images').mkdir(exist_ok=True)
# (data_dir / 'mock_data_images' / 'train').mkdir(exist_ok=True)
# (data_dir / 'mock_data_images' / 'valid').mkdir(exist_ok=True)

# (data_dir / 'mock_data_masks').mkdir(exist_ok=True)
# (data_dir / 'mock_data_masks' / 'train').mkdir(exist_ok=True)
# (data_dir / 'mock_data_masks' / 'valid').mkdir(exist_ok=True)

# for i, (image, mask) in enumerate(train_data):
#     Image.fromarray(image).save(data_dir / 'mock_data_images' / 'train' / (str(i) + '.tiff'))
#     Image.fromarray(mask).save(data_dir / 'mock_data_masks' / 'train' / (str(i) + '.tiff'))
    
# for i, (image, mask) in enumerate(valid_data):
#     Image.fromarray(image).save(data_dir / 'mock_data_images' / 'valid' / (str(i) + '.tiff'))
#     Image.fromarray(mask).save(data_dir / 'mock_data_masks' / 'valid' / (str(i) + '.tiff'))

In [17]:
data_transform = transforms.Compose([
    # TODO specify augmentations (e.g. histogram normalization)
    Augmentor(
        dict_key="features",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 256.).unsqueeze_(0).float()),
    Augmentor(
        dict_key="targets",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy()).unsqueeze_(0).float()),
])

open_fn = lambda x: {"features": x[0], "targets": x[1]}


train_loader = UtilsFactory.create_loader(
    train_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    workers=n_workers, 
    shuffle=True)

valid_loader = UtilsFactory.create_loader(
    valid_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    workers=n_workers, 
    shuffle=False)

loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = valid_loader

In [18]:
model = UNet(in_channels=1)

In [19]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# scheduler = None  # for OneCycle usage
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 40], gamma=0.3)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)

In [20]:
from catalyst.dl.callbacks import (
    ClassificationLossCallback, 
    BaseMetrics, Logger, TensorboardLogger,
    OptimizerCallback, SchedulerCallback, CheckpointCallback, 
    PrecisionCallback, OneCycleLR)

n_epochs = 50
logdir = "./logs/segmentation_notebook"

callbacks = collections.OrderedDict()

callbacks["loss"] = ClassificationLossCallback()
callbacks["optimizer"] = OptimizerCallback()
callbacks["metrics"] = BaseMetrics()

# OneCylce custom scheduler callback
callbacks["scheduler"] = OneCycleLR(
    cycle_len=n_epochs,
    div=3, cut_div=4, momentum_range=(0.95, 0.85))

# Pytorch scheduler callback
# callbacks["scheduler"] = SchedulerCallback(
#     reduce_metric="loss_main")

callbacks["saver"] = CheckpointCallback()
callbacks["logger"] = Logger()
callbacks["tflogger"] = TensorboardLogger()

In [21]:
from catalyst.dl.runner import ClassificationRunner

runner = ClassificationRunner(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler)
runner.train_stage(
    loaders=loaders, 
    callbacks=callbacks, 
    logdir=logdir,
    epochs=n_epochs, verbose=True)



0 * Epoch (train):   0% 0/63 [00:00<?, ?it/s]

SegNet(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc3): Sequential

/home/ecohen/projects/miniconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


0 * Epoch (train):   0% 0/63 [00:13<?, ?it/s, batch time=13.77238, data time=0.35612, loss_main=0.80295, lr_main=0.00033, momentum_main=0.94987, sample per second=0.29044]

0 * Epoch (train):   2% 1/63 [00:13<14:17, 13.83s/it, batch time=13.77238, data time=0.35612, loss_main=0.80295, lr_main=0.00033, momentum_main=0.94987, sample per second=0.29044]

0 * Epoch (train):   2% 1/63 [00:14<14:48, 14.33s/it, batch time=0.35206, data time=0.00704, loss_main=0.77321, lr_main=0.00033, momentum_main=0.94975, sample per second=11.36175]

0 * Epoch (train):   3% 2/63 [00:14<07:17,  7.16s/it, batch time=0.35206, data time=0.00704, loss_main=0.77321, lr_main=0.00033, momentum_main=0.94975, sample per second=11.36175]

0 * Epoch (train)

0 * Epoch (train):  33% 21/63 [00:24<00:49,  1.17s/it, batch time=0.36109, data time=0.00370, loss_main=0.70528, lr_main=0.00035, momentum_main=0.94720, sample per second=11.07754]

0 * Epoch (train):  35% 22/63 [00:24<00:45,  1.12s/it, batch time=0.36109, data time=0.00370, loss_main=0.70528, lr_main=0.00035, momentum_main=0.94720, sample per second=11.07754]

0 * Epoch (train):  35% 22/63 [00:25<00:46,  1.14s/it, batch time=0.36280, data time=0.00386, loss_main=0.70844, lr_main=0.00035, momentum_main=0.94708, sample per second=11.02530]

0 * Epoch (train):  37% 23/63 [00:25<00:43,  1.09s/it, batch time=0.36280, data time=0.00386, loss_main=0.70844, lr_main=0.00035, momentum_main=0.94708, sample per second=11.02530]

0 * Epoch (train):  37% 23/63 [00:25<00:44,  1.11s/it, batch time=0.36069, data time=0.00384, loss_main=0.70441, lr_main=0.00035, momentum_main=0.94695, sample per second=11.08976]

0 * Epoch (train):  38% 24/63 [00:25<00:41,  1.07s/it, batch time=0.36069, data time=0.003

0 * Epoch (train):  68% 43/63 [00:35<00:16,  1.22it/s, batch time=0.36390, data time=0.00563, loss_main=0.70088, lr_main=0.00037, momentum_main=0.94454, sample per second=10.99195]

0 * Epoch (train):  68% 43/63 [00:35<00:16,  1.20it/s, batch time=0.36204, data time=0.00524, loss_main=0.70167, lr_main=0.00037, momentum_main=0.94441, sample per second=11.04855]

0 * Epoch (train):  70% 44/63 [00:35<00:15,  1.23it/s, batch time=0.36204, data time=0.00524, loss_main=0.70167, lr_main=0.00037, momentum_main=0.94441, sample per second=11.04855]

0 * Epoch (train):  70% 44/63 [00:36<00:15,  1.21it/s, batch time=0.36453, data time=0.00592, loss_main=0.70138, lr_main=0.00037, momentum_main=0.94428, sample per second=10.97312]

0 * Epoch (train):  71% 45/63 [00:36<00:14,  1.24it/s, batch time=0.36453, data time=0.00592, loss_main=0.70138, lr_main=0.00037, momentum_main=0.94428, sample per second=10.97312]

0 * Epoch (train):  71% 45/63 [00:36<00:14,  1.22it/s, batch time=0.36212, data time=0.005

...

7 * Epoch (train):   0% 0/63 [00:00<?, ?it/s]

7 * Epoch (train):   0% 0/63 [00:01<?, ?it/s, batch time=0.89186, data time=0.51239, loss_main=0.76325, lr_main=0.00071, momentum_main=0.89295, sample per second=4.48501]

7 * Epoch (train):   2% 1/63 [00:01<01:04,  1.05s/it, batch time=0.89186, data time=0.51239, loss_main=0.76325, lr_main=0.00071, momentum_main=0.89295, sample per second=4.48501]

7 * Epoch (train):   2% 1/63 [00:01<01:37,  1.56s/it, batch time=0.36698, data time=0.00698, loss_main=0.79182, lr_main=0.00071, momentum_main=0.89282, sample per second=10.89966]

7 * Epoch (train):   3% 2/63 [00:01<00:47,  1.28it/s, batch time=0.36698, data time=0.00698, loss_main=0.79182, lr_main=0.00071, momentum_main=0.89282, sample per second=10.89966]

7 * Epoch (train):   3% 2/63 [00:02<01:03,  1.04s/it, batch time=0.36977, data time=0.00555, loss_main=0.75034, lr_main=0.00071, momentum_main=0.89269, sample per second=10.81739]

7 * Epoch (train):   5% 3/63 [00:02<00:41,  1.44it/s, batc

7 * Epoch (train):  35% 22/63 [00:11<00:22,  1.84it/s, batch time=0.36492, data time=0.00575, loss_main=0.70322, lr_main=0.00073, momentum_main=0.89028, sample per second=10.96122]

7 * Epoch (train):  35% 22/63 [00:12<00:23,  1.76it/s, batch time=0.36908, data time=0.00574, loss_main=0.70209, lr_main=0.00073, momentum_main=0.89015, sample per second=10.83770]

7 * Epoch (train):  37% 23/63 [00:12<00:21,  1.84it/s, batch time=0.36908, data time=0.00574, loss_main=0.70209, lr_main=0.00073, momentum_main=0.89015, sample per second=10.83770]

7 * Epoch (train):  37% 23/63 [00:13<00:22,  1.77it/s, batch time=0.36889, data time=0.00548, loss_main=0.70073, lr_main=0.00073, momentum_main=0.89003, sample per second=10.84327]

7 * Epoch (train):  38% 24/63 [00:13<00:21,  1.84it/s, batch time=0.36889, data time=0.00548, loss_main=0.70073, lr_main=0.00073, momentum_main=0.89003, sample per second=10.84327]

7 * Epoch (train):  38% 24/63 [00:13<00:22,  1.77it/s, batch time=0.36726, data time=0.004

7 * Epoch (train):  68% 43/63 [00:23<00:11,  1.80it/s, batch time=0.38967, data time=0.00564, loss_main=0.69639, lr_main=0.00075, momentum_main=0.88748, sample per second=10.26511]

7 * Epoch (train):  70% 44/63 [00:23<00:10,  1.84it/s, batch time=0.38967, data time=0.00564, loss_main=0.69639, lr_main=0.00075, momentum_main=0.88748, sample per second=10.26511]

7 * Epoch (train):  70% 44/63 [00:24<00:10,  1.80it/s, batch time=0.38361, data time=0.00613, loss_main=0.69571, lr_main=0.00075, momentum_main=0.88736, sample per second=10.42721]

7 * Epoch (train):  71% 45/63 [00:24<00:09,  1.84it/s, batch time=0.38361, data time=0.00613, loss_main=0.69571, lr_main=0.00075, momentum_main=0.88736, sample per second=10.42721]

7 * Epoch (train):  71% 45/63 [00:25<00:10,  1.80it/s, batch time=0.38967, data time=0.00450, loss_main=0.69616, lr_main=0.00075, momentum_main=0.88723, sample per second=10.26516]

7 * Epoch (train):  73% 46/63 [00:25<00:09,  1.84it/s, batch time=0.38967, data time=0.004

  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ecohen/projects/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/ecohen/projects/miniconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeou

KeyboardInterrupt: 